# HiHydroSoil to SPHY
## Juncal Norte - august, 2022
#### Paul Sandoval Quilodrán - https://github.com/SQPaul
#### Run in QGIS

### Import packages

In [1]:
import glob
import rasterio
import numpy as np
from matplotlib import pyplot
from rasterio.plot import show

## Selecting useful data

In [ ]:
hydrosoil = glob.glob("/media/phi/SANDOVAL/Juncal_norte/SOIL_PROPERTIES/*.tif") #r"/media/phi/SANDOVAL/LAST PAUL/VI año/Memoria de título/Modelación/Soil_properties#/*.tif" 

for i in hydrosoil:
    name = i[49:-8]
    #print(name)
    if name == "WCpF2_M_250m_TOPSOIL":
        R_FC = i
    elif name == "WCsat_M_250m_TOPSOIL":
        R_SC = i
    elif name == "WCpF42_M_250m_TOPSOIL":
        R_PWP = i
    elif name == "WCpF3_M_250m_TOPSOIL":
        R_WP = i
    elif name == "Ksat_M_250m_TOPSOIL":
        R_KSAT = i
    elif name == "WCpF2_M_250m_SUBSOIL":
        S_FC = i
    elif name == "WCsat_M_250m_SUBSOIL":
        S_SC = i
    elif name == "Ksat_M_250m_SUBSOIL":
        S_KSAT = i
        
hydrosoil_sel = [R_FC,R_SC,R_PWP,R_WP,R_KSAT,S_FC,S_SC,S_KSAT] 

## Resampling HiHydroSoil 
### Change crs, resolution and extent
#### Run in QGIS


In [ ]:
for i in hydrosoil_sel:
    output = i[:-4]+"_mod.tif"
    processing.run("gdal:warpreproject", 
               {'INPUT':str(i),
                'SOURCE_CRS':None,
                'TARGET_CRS':QgsCoordinateReferenceSystem('EPSG:32719'),
                'RESAMPLING':0,'NODATA':None,
                'TARGET_RESOLUTION':500,
                'OPTIONS':'',
                'DATA_TYPE':0,
                'TARGET_EXTENT':'380681.000000000,413181.000000000,6339419.000000000,6381419.000000000 [EPSG:32719]',
                'TARGET_EXTENT_CRS':None,
                'MULTITHREADING':False,
                'EXTRA':'',
                'OUTPUT':str(output)})

## Multiply the rasters by 0.0001 and write it in PCRaster format

In [2]:
hydrosoil = glob.glob("/media/phi/SANDOVAL/Juncal_norte/SOIL_PROPERTIES/*mod.tif") #r"/media/phi/SANDOVAL/LAST PAUL/VI año/Memoria de título/Modelación/Soil_properties#/*.tif" 

for i in hydrosoil:
    name = i[49:-12]
    #print(name)
    if name == "WCpF2_M_250m_TOPSOIL":
        R_FC = i
    elif name == "WCsat_M_250m_TOPSOIL":
        R_SC = i
    elif name == "WCpF42_M_250m_TOPSOIL":
        R_PWP = i
    elif name == "WCpF3_M_250m_TOPSOIL":
        R_WP = i
    elif name == "Ksat_M_250m_TOPSOIL":
        R_KSAT = i
    elif name == "WCpF2_M_250m_SUBSOIL":
        S_FC = i
    elif name == "WCsat_M_250m_SUBSOIL":
        S_SC = i
    elif name == "Ksat_M_250m_SUBSOIL":
        S_KSAT = i
        
hydrosoil_sel = [R_FC,R_SC,R_PWP,R_WP,R_KSAT,S_FC,S_SC,S_KSAT] 

In [3]:
hydrosoil_names = ["R_FC","R_SC","R_PWP","R_WP","R_KSAT","S_FC","S_SC","S_KSAT"]

for i in range(len(hydrosoil_names)):
    output = "/media/phi/SANDOVAL/Juncal_norte/SOIL_PROPERTIES/"+hydrosoil_names[i]+".tif"
    raster = rasterio.open(hydrosoil_sel[i])
    raster_array = raster.read(1)*0.0001
    new_raster = rasterio.open(
        str(output),"w",
        driver = "PCRaster",
        pcraster_valuescale = "VS_SCALAR",
        height = raster.shape[0],
        width = raster.shape[1],
        count = 1,
        nodata = raster.nodata,
        dtype = np.float32,
        crs = 32719,
        transform = raster.transform)
    new_raster.write(raster_array,1)
    new_raster.close()